In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
each_words=[]
batches=[]
files=[]
neglect=["?","!"]
data_file=open("D:/deva/Intents new data.json").read()
data=json.loads(data_file)
data

{'intents': [{'intent': 'Greeting',
   'text': ['Hi',
    'Hi there',
    'Hola',
    'Hello',
    'Hello there',
    'Hya',
    'Hya there'],
   'responses': ['Hi human, please tell me your GeniSys user',
    'Hello human, please tell me your GeniSys user',
    'Hola human, please tell me your GeniSys user'],
   'extension': {'function': '', 'entities': False, 'responses': []},
   'context': {'in': '', 'out': 'GreetingUserRequest', 'clear': False},
   'entityType': 'NA',
   'entities': []},
  {'intent': 'GreetingResponse',
   'text': ['My user is Adam',
    'This is Adam',
    'I am Adam',
    'It is Adam',
    'My user is Bella',
    'This is Bella',
    'I am Bella',
    'It is Bella'],
   'responses': ['Great! Hi <HUMAN>! How can I help?',
    'Good! Hi <HUMAN>, how can I help you?',
    'Cool! Hello <HUMAN>, what can I do for you?',
    'OK! Hola <HUMAN>, how can I help you?',
    'OK! hi <HUMAN>, what can I do for you?'],
   'extension': {'function': 'extensions.gHumans.updateHum

In [4]:
for content in data["intents"]:
    for tex in content["text"]:
        w=nltk.word_tokenize(tex)
        each_words.extend(w)
        files.append((w,content["intent"]))
        if content["intent"] not in batches:
            batches.append(content["intent"])        

In [5]:
each_words=[lemmatizer.lemmatize(w.lower()) for w in each_words if w not in neglect]
each_words=sorted(list(set(each_words)))
batches=sorted(list(set(batches)))
print(len(files),"files")
print(len(batches),"batches",batches)
print(len(each_words),"Each Words are Lemmatized",each_words)
pickle.dump(each_words,open("each_words.pkl","wb"))
pickle.dump(batches,open("batches.pkl","wb"))

143 files
22 batches ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
117 Each Words are Lemmatized ["'s", ',', 'a', 'about', 'adam', 'adios', 'am', 'any', 'anyone', 'are', 'aware', 'bay', 'be', 'bella', 'bored', 'by', 'bye', 'call', 'camera', 'can', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'helpful', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identify', 'in', 'intelligent', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'open

In [6]:
training_data=[]
output_empty=[0]*len(batches)
for doc in files:
    bag=[]
    pattern_words = doc[0]
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in each_words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_empty)
    output_row[batches.index(doc[1])]=1
    training_data.append([bag,output_row])
random.shuffle(training_data)
training_data=np.array(training_data)
x=list(training_data[:,0])
y=list(training_data[:,1])
print("Training data")

Training data


In [7]:
model=Sequential()
model.add(Dense(244,input_shape=(len(x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]),activation="softmax"))
sgd=SGD(momentum=0.9,nesterov=True,lr=0.01, decay=1e-6)
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer=sgd)
hist=model.fit(np.array(x),np.array(y),epochs=200,batch_size=5,verbose=1)
print("Machine Ceated")

Epoch 1/200
29/29 [==============================] - 0s 1ms/step - loss: 3.0647 - accuracy: 0.0490
Epoch 2/200
29/29 [==============================] - 0s 1ms/step - loss: 2.9097 - accuracy: 0.1678
Epoch 3/200
29/29 [==============================] - 0s 1ms/step - loss: 2.7071 - accuracy: 0.2727
Epoch 4/200
29/29 [==============================] - 0s 948us/step - loss: 2.4835 - accuracy: 0.3776
Epoch 5/200
29/29 [==============================] - 0s 862us/step - loss: 2.1533 - accuracy: 0.4406
Epoch 6/200
29/29 [==============================] - 0s 948us/step - loss: 1.8200 - accuracy: 0.5524
Epoch 7/200
29/29 [==============================] - 0s 1ms/step - loss: 1.5876 - accuracy: 0.5874
Epoch 8/200
29/29 [==============================] - 0s 862us/step - loss: 1.3866 - accuracy: 0.5874
Epoch 9/200
29/29 [==============================] - 0s 1ms/step - loss: 1.1759 - accuracy: 0.7133
Epoch 10/200
29/29 [==============================] - 0s 948us/step - loss: 0.9902 - accuracy: 0.7483

29/29 [==============================] - 0s 776us/step - loss: 0.0522 - accuracy: 0.9860
Epoch 82/200
29/29 [==============================] - 0s 776us/step - loss: 0.0341 - accuracy: 0.9860
Epoch 83/200
29/29 [==============================] - 0s 862us/step - loss: 0.0361 - accuracy: 0.9930
Epoch 84/200
29/29 [==============================] - 0s 345us/step - loss: 0.0372 - accuracy: 0.9930
Epoch 85/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 86/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0169 - accuracy: 1.0000
Epoch 87/200
29/29 [==============================] - 0s 538us/step - loss: 0.0254 - accuracy: 0.9930
Epoch 88/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0254 - accuracy: 0.9930
Epoch 89/200
29/29 [==============================] - 0s 624us/step - loss: 0.0428 - accuracy: 0.9860
Epoch 90/200
29/29 [==============================] - 0s 969us/step - loss: 0.0369 - accuracy: 1.0000

29/29 [==============================] - 0s 690us/step - loss: 0.0085 - accuracy: 1.0000
Epoch 162/200
29/29 [==============================] - 0s 690us/step - loss: 0.0088 - accuracy: 1.0000
Epoch 163/200
29/29 [==============================] - 0s 776us/step - loss: 0.0082 - accuracy: 1.0000
Epoch 164/200
29/29 [==============================] - 0s 538us/step - loss: 0.0137 - accuracy: 1.0000
Epoch 165/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0132 - accuracy: 1.0000
Epoch 166/200
29/29 [==============================] - 0s 776us/step - loss: 0.0134 - accuracy: 1.0000
Epoch 167/200
29/29 [==============================] - 0s 538us/step - loss: 0.0236 - accuracy: 1.0000
Epoch 168/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 169/200
29/29 [==============================] - 0s 776us/step - loss: 0.0193 - accuracy: 0.9860
Epoch 170/200
29/29 [==============================] - 0s 538us/step - loss: 0.0059 - accur

In [8]:
model.save("ChatBot Machine.h5",hist)

In [9]:
from keras.models import load_model
model=load_model("ChatBot Machine.h5")
data=json.loads(open("D:/deva/Intents new data.json").read())
each_words=pickle.load(open("each_words.pkl","rb"))
batches=pickle.load(open("batches.pkl","rb"))

In [10]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence,each_words,show_details=True):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(each_words)
    for s in sentence_words:
        for i,w in enumerate(each_words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("Found in bag:%s" %w)
    return(np.array(bag)) 

def Prediction(sentence,model):
    p=bow(sentence,each_words,show_details=False)
    res=model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": batches[r[0]], "probability": str(r[1])})
    return return_list


def Reply(ints,intents_json):
    r=ints[0]["intent"]
    list_intents=intents_json["intents"]
    for i in list_intents:
        if(i["intent"]==r):
            result=random.choice(i["responses"])
    return result

def Robot_Response(msg):
    ints=Prediction(msg,model)
    res=Reply(ints,data)
    return res
        
    

In [11]:
import tkinter
from tkinter import *
def send():
    Messege = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if Messege != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + Messege + '\n\n')
        ChatLog.config(foreground="white", font=("Calibri", 12 ))
        Result = Robot_Response(Messege)
        ChatLog.insert(END, "Bot: " + Result + '\n\n')
        ChatLog.config(foreground="white", font=("Calibri", 12,'bold' ))
        ChatLog.yview(END)
base = Tk()
base.title("Chat With Robot")
base.geometry("600x600")
base.resizable(width=FALSE, height=FALSE)
ChatLog = Text(base, bd=0, bg="#87ceeb", height="8", width="50", font="Calibri",)
ChatLog.config(state=DISABLED)
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="arrow")
ChatLog['yscrollcommand'] = scrollbar.set
SendButton = Button(base, font=("Calibri",15,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Calibri")
scrollbar.place(x=576,y=6, height=586)
ChatLog.place(x=6,y=6, height=586, width=570)
EntryBox.place(x=6, y=551, height=51, width=443)
SendButton.place(x=450, y=551, height=50)
base.mainloop()